# Classification of Lab Samples

In [1]:
import init
from common import constants as cn
from common.trinary_data import TrinaryData
from common.data_provider import DataProvider
from common_python.plots import util_plots
from common_python.classifier import classifier_ensemble
from common_python.classifier import classifier_collection
from common_python.classifier.classifier_ensemble_random_forest import ClassifierEnsembleRandomForest
from common import transform_data

import collections
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score

%matplotlib inline

IndentationError: unexpected indent (transform_data.py, line 46)

In [ ]:
data = TrinaryData()
data.df_X.head()

In [ ]:
STOP HERE

In [ ]:
df_sampleAM = transform_data.trinaryReadsDF(
    csv_file="AM_MDM_Mtb_transcripts_DEseq.csv")
df_sampleAW = transform_data.trinaryReadsDF(
"AW_plus_v_AW_neg_Mtb_transcripts_DEseq.csv")

In [ ]:
df_sampleAM = df_sampleAM.T
df_sampleAM.head()

In [ ]:
df_sampleAW = df_sampleAW.T
df_sampleAW.head()

## Classification Validations
Classify T2-T25 and see if result is same as original class


In [ ]:
provider = DataProvider(is_normalize=False)
provider.do()

In [ ]:
def getTimeSample(time_index):
    df0 = provider.dfs_data[0]
    num = len(provider.dfs_data)
    ser = pd.Series(np.repeat(0, len(df0.index)), index=df0.index)
    for idx in range(num):
        ser += provider.dfs_data[idx][time_index]
    df = pd.DataFrame(ser/num)
    df_result = transform_data.trinaryReadsDF(df_sample=df)
    return df_result.T

In [ ]:
getTimeSample(2).head()

In [ ]:
provider.dfs_data[1].head()

In [ ]:
svm_ensemble = classifier_ensemble.ClassifierEnsemble(
        classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=15, size=30)
df_X = data.df_X.copy()
df_X.columns = data.features
svm_ensemble.fit(df_X, data.ser_y)

In [ ]:

prediced_classesp = [svm_ensemble.predict(getTimeSample(n)) for n in [24, 25]]

In [ ]:
dfff = pd.concat([dff, *prediced_classesp])
dfff.index = range(2, 26)
dfff

In [ ]:
dff = pd.concat(prediced_classes)
dff.index = [2 + v for v in range(len(prediced_classes))]
dff

In [ ]:
prediced_classes[0]

In [ ]:
data.ser_y
df_expected = pd.DataFrame(data.ser_y)
df_expected.columns = ["columns"]
df_expected["values"] = 1.0
dff_expected = df_expected.pivot(columns="columns", values="values")
dff_expected = dff_expected.fillna(0)
dff_expected.index = [int(v[1:]) for v in dff_expected.index]
dff_expected 
df_diff = (dff_expected - dfff)
df_diff = df_diff.applymap(lambda v: np.abs(v))
df_diff.sum().sum()/len(df_diff)

## Classification of Lab Samples

In [ ]:
svm_ensemble = classifier_ensemble.ClassifierEnsemble(
        classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=15, size=30)
df_X = data.df_X.copy()
df_X.columns = data.features
svm_ensemble.fit(df_X, data.ser_y)

In [ ]:
svm_ensemble.predict(df_sampleAM)

In [ ]:
svm_ensemble.predict(df_sampleAW)

## Comparisons With Random Forest

In [ ]:
dummy_columns = [v+"-" for v in data.features]
truncated_columns = [f[0:f.index("-")] for f in dummy_columns]
len(truncated_columns)
df_X = data.df_X.copy()
df_X.columns = truncated_columns

In [ ]:
clf = ClassifierEnsembleRandomForest(size=150, filter_high_rank=30)
clf.fit(df_X, data.ser_y)

In [ ]:
df_sampleAM[truncated_columns].head()

In [ ]:
clf.predict(df_sampleAM[clf.columns])

In [ ]:
clf.predict(df_sampleAW[truncated_columns])

## Random Forest Validations

In [ ]:
set(clf.columns).difference(data.features)

In [ ]:
df_X = data.df_X.copy()
df_X.columns = data.features
df_X[clf.columns]

In [ ]:
def calcCVR(num_features, num_classifiers, num_iterations=10):
     clf = ClassifierEnsembleRandomForest(
         filter_high_rank=num_features, size=num_classifiers)
     return classifier_collection.ClassifierCollection.crossValidateByState(
         clf, df_X, data.ser_y, num_iterations)

In [ ]:
calcCVR(30, 100)